In [1]:
# !pip install --upgrade datasets -q
# !pip install jiwer -q
# !pip install evaluate -q

In [2]:
from datasets import load_dataset, DatasetDict
from torch.utils.data import DataLoader
import torch
from torchaudio import load
import evaluate

# Load Whisper processor
from transformers import WhisperProcessor, WhisperFeatureExtractor, WhisperTokenizer
from transformers import WhisperForConditionalGeneration
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

from dotenv import load_dotenv
from huggingface_hub import login
import os


In [3]:

load_dotenv()
login_token = os.getenv('HuggingFaceToken')

login(login_token)

## Data Modeling

In [4]:
common_voice = DatasetDict()

common_voice["train"] = load_dataset(
    "mozilla-foundation/common_voice_17_0", "ml", split="train+validation",
)
common_voice["test"] = load_dataset(
    "mozilla-foundation/common_voice_17_0", "ml", split="test",
)

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 2023
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 710
    })
})


In [5]:
# remove unwanted features
common_voice = common_voice.select_columns(['audio', 'sentence'])

In [6]:
print(common_voice['train'][0]['audio'])

{'path': 'C:\\Users\\VICTUS\\.cache\\huggingface\\datasets\\downloads\\extracted\\3e7b12b0fa0deddeccc4a37a644801109d30fe7dda8b39a953688d0be0744a2f\\ml_train_0/common_voice_ml_37003897.mp3', 'array': array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
       1.33694380e-06, 6.72575652e-07, 1.44025307e-07], shape=(150336,)), 'sampling_rate': 48000}


In [7]:
# 48kHz -> 16kHz
from datasets import Audio
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [8]:
common_voice['train'][0]['audio']['array']

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
       -1.61271237e-06, -1.26397367e-06,  1.32478658e-06], shape=(50112,))

In [9]:
# def collate_fuc(batch):
#     print(len(batch))
#     print(batch[0].keys())
# data_loader = DataLoader(common_voice['train'], batch_size=3, shuffle=True, collate_fn=collate_fuc)

In [10]:
# for i in data_loader:
#     break

In [11]:
# filtering audio len > 30 sec

In [12]:
# whisper proccessor wrap whisperFeature extractor for audio and whispertokenizer for text labels as one processor
feature_extractor = WhisperFeatureExtractor.from_pretrained('openai/whisper-small', task='transcribe', language='malayalam')
tokenizer = WhisperTokenizer.from_pretrained('openai/whisper-small', task='transcribe', language='malayalam')
processor = WhisperProcessor.from_pretrained('openai/whisper-small', task='transcribe', language='malayalam')

In [13]:
# sample_tokens = tokenizer.encode('ഇല്ല മോനേ')
# tokenizer.decode(sample_tokens)

In [14]:
text = 'ഇല്ല മോനേ'
batch = processor(text=text, sampling_rate=16000)
processor.tokenizer.decode(batch['input_ids'])

'<|startoftranscript|><|ml|><|transcribe|><|notimestamps|>ഇല്ല മോനേ<|endoftext|>'

In [15]:
# Prepare data
def prepare_data(batch):
    # processor have both feature extractor for audio and tokenizer for text, so we just pass both of theem
    batch = processor(audio=batch['audio']['array'],
                      text=batch['sentence'],
                      sampling_rate=processor.feature_extractor.sampling_rate)
    return batch

In [16]:
MAX_LABEL_TOKEN_WHISPER_SUPPORT = 448
def filter_label_token(batch):
    return len(batch['labels']) <= MAX_LABEL_TOKEN_WHISPER_SUPPORT

In [17]:
common_voice = common_voice.map(prepare_data, batched=False)

In [18]:
common_voice = common_voice.filter(filter_label_token, batched=False)

In [19]:
common_voice = common_voice.select_columns(['input_features', 'labels'])

In [20]:
common_voice

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 2022
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 710
    })
})

In [21]:
torch.tensor(common_voice['train'][0]['input_features']).shape # (1, 80, 3000)

torch.Size([1, 80, 3000])

Dataloader takes random datapoints, here it will look like {input_feature:.., labels}, when batch enabled it will be like [{inp:.., lable:..}, {inp: .., label:..}], we need to use data collator for pad them and join them

In [22]:
# feature_extractor.pad(common_voice['train'][:2])

In [23]:
len(common_voice['train'][0]['labels'])

25

In [24]:
# Data Collator for padding

class DataCollatorForSeqToSeqPadding:
    def __init__(self, processor: WhisperProcessor):
        self.processor = processor

    def __call__(self, batch):
        # batch = [ {'input_feature':[], labels:[]}, {} ...]
        input_features = [{"input_features" : data['input_features'][0]} for data in batch]
        labels = [{"input_ids" : data['labels']} for data in batch]

        # feature extractor from hugging face already support padding to {'input_features':[]}
        # padding using feature extractor for audio and tokenizer for labels
        batch = self.processor.feature_extractor.pad(input_features, return_tensors='pt') # created a batch object , later will add label to this too, that's how huggingface model expect data {'input_features':[], labels:[]}

        # whisper tokenizer.pad will check the {'input_ids':[]} for padding and return in same forma
        labels = self.processor.tokenizer.pad(labels, return_tensors='pt')


        # since we are using hugging face model we don't need to stack the tensor cuz the hugging face (whisper here) model expect input like {'input_features':[], labels:[]}
        # tensor stacking
        # input_features = torch.stack(input_features, dim=0)
        # labels = torch.stack(labels) # have


        labels = labels['input_ids'].masked_fill(labels['attention_mask'].eq(0), -100)

        # we are removing the start token since the hugging face model design to automatically add start token
        # by doing shifting labels to right [1, 2, <\s>] -> [<s>, 1, 2], where we using this shifted tensor as input
        # and the non shifted as the labels to calculate the loss (the model gets what's his start token from the config)

        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            print()
            labels = labels[:, 1:]

        batch["labels"] = labels


        return batch


In [25]:
tokenizer.bos_token_id

50257

In [26]:
tokenizer.decode(processor.tokenizer.bos_token_id)

'<|endoftext|>'

In [27]:
collate_fn = DataCollatorForSeqToSeqPadding(processor=processor)

# data loader for just checking the data collator, seqtoseq trainer does not need dataloader (inbuilt)
data_loader = DataLoader(dataset=common_voice['train'],
                         collate_fn=collate_fn,
                         batch_size=2,
                         shuffle=True,
                         drop_last=True)

In [28]:
for batch in data_loader:
    print(batch.keys()) # torch.Size([2, 80, 3000]), Yes now it's coming as batch size and not in (2, 1, 80, 3000)
    break

dict_keys(['input_features', 'labels'])


## Model setup

In [29]:
metric = evaluate.load('wer')

In [30]:
processor.tokenizer.pad_token_ids

50257

In [31]:
def compute_metrics(pred):
    # pred will look like {'label_ids':[torch.tensor], prediction:[torch.tensor]}
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # change all -100 value which we set for loss calculation back to padding since we are calculating wer
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # convert to string and remove the padding token if there is.. if it was -100 then it won't work that is why we changed back to padding
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    # Now compute, the metric comes from the evaluate and we set it in the arugment of Trainer so compute metric use this metric here
    wer = metric.compute(predictions=pred_str, references=label_str)

    return {'wer': wer} # standard form of hugging face

### Lora Setup

In [32]:
from peft import LoraConfig, TaskType
from peft import get_peft_model

In [33]:
# Choose model size here
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

# forced decoder ids automatically add tokens at specified position (1, tokenizer.bos_token), so at decoder time the model automaticall generate it
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [34]:
lora_r = 64
lora_alpha = 64
learning_rate = 5e-5
peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=0.01,
    bias='none',
    target_modules=['q_proj', 'v_proj', 'out_proj'],
    task_type=None,
)

In [35]:
model.config.use_cache = False

In [36]:
model.enable_input_require_grads()

In [37]:
model = get_peft_model(model, peft_config)

In [38]:
model.print_trainable_parameters()

trainable params: 10,616,832 || all params: 252,351,744 || trainable%: 4.2072


In [39]:
from transformers import Seq2SeqTrainingArguments

In [40]:
training_args = Seq2SeqTrainingArguments(output_dir='checkpoints',
                                         eval_strategy='steps',
                                         learning_rate=learning_rate,
                                         gradient_checkpointing=True,
                                         per_device_train_batch_size=4,
                                         gradient_accumulation_steps=4,
                                         warmup_steps=20,
                                         predict_with_generate=True,
                                         generation_max_length=30,
                                         per_device_eval_batch_size=1,
                                        #  eval_accumulation_steps=2,
                                         fp16=True,
                                         save_steps=100,
                                         eval_steps=10,
                                         logging_dir=f'runs/{lora_r}_{lora_alpha}_{learning_rate}',
                                         report_to=['tensorboard'],
                                         load_best_model_at_end=True,
                                         metric_for_best_model='wer',
                                         num_train_epochs=5,
                                         torch_empty_cache_steps=5,
                                         dataloader_drop_last=True,
                                        #  dataloader_num_workers=2,
                                        #  dataloader_pin_memory=True,
                                         logging_strategy='steps',
                                         logging_steps=10,
                                         optim='adamw_torch',
                                         label_names=['labels'],
                                         lr_scheduler_type="cosine_with_restarts")

In [41]:
metric.compute(predictions=['hello', 'world'], references=['hello', 'there'])

0.5

In [42]:
trainer = Seq2SeqTrainer(
    model=model,
    train_dataset=common_voice['train'],
    eval_dataset=common_voice['test'].shuffle(seed=0).select(range(20)),
    args=training_args,
    data_collator=DataCollatorForSeqToSeqPadding(processor=processor),
    compute_metrics=compute_metrics
)

In [43]:
torch.cuda.empty_cache()

In [44]:
trainer.train(resume_from_checkpoint="checkpoints\\checkpoint-300")

d:\Machine Learning\INTERNSHIP\Whisper-Malayalam-Finetuning\.venv\Lib\site-packages\torch\utils\checkpoint.py:86: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
310,0.754500,0.824279,1.024096
320,0.755900,0.809989,1.024096
330,0.731100,0.797589,1.012048
340,0.732700,0.787002,1.024096
350,0.729700,0.771253,1.012048
360,0.717600,0.766534,1.036145
370,0.701100,0.761560,1.036145
380,0.692500,0.745442,0.987952
390,0.703600,0.735520,1.000000
400,0.680000,0.720058,1.012048


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
d:\Machine Learning\INTERNSHIP\Whisper-Malayalam-Finetuning\.venv\Lib\site-packages\torch\utils\checkpoint.py:86: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
d:\Machine Learning\INTERNSHIP\Whisper-Malayalam-Finetuning\.venv\Lib\site-packages\torch\utils\checkpoint.py:86: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  wa

TrainOutput(global_step=634, training_loss=0.3437549063835986, metrics={'train_runtime': 1748.3949, 'train_samples_per_second': 5.782, 'train_steps_per_second': 0.363, 'total_flos': 3.06912374784e+18, 'train_loss': 0.3437549063835986, 'epoch': 5.0})

In [45]:
# Inference
from peft import AutoPeftModel
from transformers import pipeline

In [47]:
test_model = AutoPeftModel.from_pretrained('checkpoints\\checkpoint-600')

In [48]:
test_model.to('cuda');

In [49]:
pipe = pipeline(
    task='automatic-speech-recognition',
    model=test_model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor
)

Device set to use cuda:0


In [50]:
pipe('record_out.wav')

d:\Machine Learning\INTERNSHIP\Whisper-Malayalam-Finetuning\.venv\Lib\site-packages\transformers\models\whisper\generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


{'text': 'ആലോ എന്നെ പര്രവാജ്ത്ത് ഞാന് കുണ്ട്ടിലുന്നുന്നും പുന്ന്നുപ്പ്പ്പ്പ്പ്പ്പുല്ല് എന്ന്നു സമുമമുനുനുനു ന്നുനുനുനുനുനുനുനുനുനുനുനുനുനുനുനുനുനുനുനുനുനുനുനുനുനുനുനുനുനുനുനുനുനുനനുനുനുനുനുന�ുന�ു�'}

In [ ]:
test_input = common_voice['test'][100]['input_features']
test_labels = common_voice['test'][100]['labels']

In [ ]:
test_input = torch.tensor(test_input).to('cuda')

In [ ]:
processor.decode(test_model.generate(input_features=test_input, task='transcribe', langauge='ml')[0])

You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.


ValueError: The following `model_kwargs` are not used by the model: ['langauge'] (note: typos in the generate arguments will also show up in this list)

In [ ]:
test_model.generate('')

AttributeError: 'str' object has no attribute 'shape'

In [ ]:
def length_of_labels(batch):
    return {"label_len" : len(batch['labels'])}

In [ ]:
df = common_voice.map(length_of_labels)

In [ ]:
train_label_length = df['train']['label_len']
test_label_lenght = df['test']['label_len']

In [ ]:
processor.tokenizer.pad([{'input_ids':df['train'][0]['labels']}])

In [ ]:
processor.tokenizer.pad()